configuracion inicial de la red

In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras.callbacks import EarlyStopping

################ CONSTANTES #################
#### modificar en caso que sea necesario ####

#ruta a la estructura de directorio de entrenamiento y test respectivamente, 
#la misma debe contener una carpeta para cada clase 
trainPath = './Data/Train'
testPath =  './Data/Test'

# tamaño de las imagenes redimensionadas
img_dim= (128, 128)

#color de la imagen (grayscale o rgb)
#los canales se setean automaticamente
color= 'rgb'
if (color=='grayscale'): canales = 1
if (color=='rgb'): canales = 3

#numero maximo de epocas
epochs = 20
earlystop = EarlyStopping(monitor='val_loss', patience=2, min_delta = 0.005, verbose=1)

#mini_batch de aprendizaje
batch_size = 16
#cantidad de imagenes generadas con la funcion flow
batch_aug = 20 

#cantidad total de muestras de entrenamiento y test
train_samples = 39209
test_samples = 12630

#setea la forma dependiendo de la configuracion de keras
if K.image_data_format() == 'channels_first':
    input_shape = (canales, img_dim[0], img_dim[1])
else:
    input_shape = (img_dim[0], img_dim[1], canales)
                                      
# data augmentation
train_datagen = ImageDataGenerator(rescale=1. / 255,
                                    rotation_range=10,
                                    width_shift_range=0.05,
                                    height_shift_range=0.05,
                                    shear_range=0.05,
                                    #zoom_range=0.1,
                                    fill_mode='nearest')

# data augmentation:
# solo reescalamiento
test_datagen = ImageDataGenerator()

#generadores de entrenamiento y test
train_generator = train_datagen.flow_from_directory(trainPath, 
                                                    target_size = img_dim,
                                                    color_mode = color,
                                                    batch_size=batch_aug)

test_generator = train_datagen.flow_from_directory(testPath, 
                                                    target_size = img_dim,
                                                    color_mode = color,
                                                    batch_size=1)

Using TensorFlow backend.


Found 39209 images belonging to 43 classes.
Found 12630 images belonging to 43 classes.


In [2]:
from keras.models import Model, Input
from keras.layers import Convolution2D, MaxPooling2D, AveragePooling2D 
from keras.layers import Dense, Dropout
from keras.layers import Activation, Flatten, merge
from keras.layers.merge import concatenate
from keras import regularizers

Definicion del modelo usando el modo sequential de keras

In [3]:
from keras.models import Sequential
from keras.layers import Conv2D
model = Sequential()
model.add(Conv2D(16, (5, 5), 
                 padding = 'same',
                 input_shape=input_shape, 
                 kernel_initializer='glorot_normal', 
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Conv2D(32, (5, 5), 
                 padding = 'same',
                 input_shape=input_shape, 
                 kernel_initializer='glorot_normal', 
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Conv2D(64, (5, 5), 
                 padding = 'same',
                 input_shape=input_shape, 
                 kernel_initializer='glorot_normal', 
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Flatten())
model.add(Dense(1024, kernel_initializer='glorot_normal', activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(512, kernel_initializer='glorot_normal', activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(43, kernel_initializer='glorot_normal', activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 128, 128, 16)      1216      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 42, 42, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 42, 42, 32)        12832     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 64)        51264     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 3136)              0         
__________

In [ ]:


model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit_generator(
    train_generator,
    steps_per_epoch=train_samples // batch_size,
    epochs=epochs,
    validation_data=test_generator,
    validation_steps=test_samples // batch_size,
    callbacks = [earlystop],
    verbose = 1)

Epoch 1/20
2450/2450 [==============================] - 1815s - loss: 1.7528 - acc: 0.4688 - val_loss: 0.7018 - val_acc: 0.7731
Epoch 2/20
2450/2450 [==============================] - 1821s - loss: 0.3847 - acc: 0.8803 - val_loss: 0.5779 - val_acc: 0.8504
Epoch 3/20
2450/2450 [==============================] - 1843s - loss: 0.2250 - acc: 0.9334 - val_loss: 0.4361 - val_acc: 0.8948
Epoch 4/20
2450/2450 [==============================] - 1863s - loss: 0.1888 - acc: 0.9463 - val_loss: 0.4202 - val_acc: 0.8973
Epoch 5/20
2450/2450 [==============================] - 1830s - loss: 0.1634 - acc: 0.9540 - val_loss: 0.2711 - val_acc: 0.9290
Epoch 6/20
2450/2450 [==============================] - 1838s - loss: 0.1504 - acc: 0.9583 - val_loss: 0.2565 - val_acc: 0.9430
Epoch 7/20
2450/2450 [==============================] - 2547s - loss: 0.1434 - acc: 0.9618 - val_loss: 0.2994 - val_acc: 0.9316
Epoch 8/20
2450/2450 [==============================] - 1882s - loss: 0.1393 - acc: 0.9644 - val_loss: 0

In [8]:
model.save('./redNeuronal_AL-AM.h5')